In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
focal_signals = []
non_focal_signals = []

path = "/content/drive/MyDrive/BERNBarcelona/"

for i in range(1, 3751):
    # Construct the filename
    filename = f"Data_F_Ind{i:04}.txt"
    file_path = path + filename

    # Load the data using comma as the delimiter
    signal = np.loadtxt(file_path, delimiter=',')
    focal_signals.append(signal)

for i in range(1, 3751):
    # Construct the filename
    filename = f"Data_N_Ind{i:04}.txt"
    file_path = path + filename

    # Load the data using comma as the delimiter
    signal = np.loadtxt(file_path, delimiter=',')
    non_focal_signals.append(signal)

In [ ]:
# Extract the desired row as a pandas Series
row = non_focal_signals[0] # Non focal signal

# Plot the line chart
fig = plt.figure(figsize=(20,4))
plt.axhline(0, color='grey')
plt.plot(row)
plt.xlabel('Time')
plt.ylabel('Intensity')
plt.title('Non Focal signal')
plt.show()

In [ ]:
# Extract the desired row as a pandas Series
row = focal_signals[0] #focal signal

# Plot the line chart
fig = plt.figure(figsize=(20,4))
plt.axhline(0, color='grey')
plt.plot(row)
plt.xlabel('Time')
plt.ylabel('Intensity')
plt.title('Focal signal')
plt.show()

In [ ]:
from scipy.signal import butter, lfilter

eeg_signals = non_focal_signals + focal_signals

fs_original = 1024
fs_downsampled = 512
lowcut = 0.5
highcut = 150
order = 4

# Downsample the EEG signals to 512 Hz (if originally sampled at 1024 Hz)
# Replace 'fs_original' with the actual original sampling rate of the EEG signals
downsample_factor = fs_original // fs_downsampled
downsampled_eeg_signals = [signal[::downsample_factor] for signal in eeg_signals]

# Apply the filter to the downsampled data using lfilter
filtered_eeg_signals = []
for signal in downsampled_eeg_signals:
    # Compute the filter coefficients for the downsampled signal's sampling rate
    nyquist_downsampled = 0.5 * fs_downsampled
    low_downsampled = lowcut / nyquist_downsampled
    high_downsampled = highcut / nyquist_downsampled
    b_downsampled, a_downsampled = butter(order, [low_downsampled, high_downsampled], btype='band')

    # Apply the filter using lfilter
    filtered_signal = lfilter(b_downsampled, a_downsampled, signal)
    filtered_eeg_signals.append(filtered_signal)

# Rereference the signals against the median of all the channels
median_reference = np.median(filtered_eeg_signals, axis=0)
re_referenced_eeg_signals = [signal - median_reference for signal in filtered_eeg_signals]


In [ ]:
# converting into a numpy array
eeg_signals = np.array(re_referenced_eeg_signals)
print(eeg_signals.shape)

In [ ]:
# Create the labels
labels = np.zeros((eeg_signals.shape[0],), dtype=np.int32)
labels[:3750] = 0 # set the first 200 rows to class 0 -> (non focal signals)
labels[3750:] = 1 # set the next 200 rows to class 1 -> (focal signals)
print(labels.shape)

In [ ]:
# Shuffle the data and labels
perm = np.random.permutation(len(eeg_signals))
data = eeg_signals[perm]
labels = labels[perm]

# saving the shuffled data and labels as numpy arrays to a file
np.save("data.npy", data)
np.save("labels.npy",labels)

print(data)
print(labels)

In [ ]:
import numpy as np

# Load data
data = np.load('data.npy')

# Load labels
labels = np.load('labels.npy')

# Split the data into training and test sets (80:20)
train_size = int(0.8 * len(data))
train_data, train_labels = data[:train_size], labels[:train_size]
test_data, test_labels = data[train_size:], labels[train_size:]

# Reshape the data for use with LSTM, GRU, and Conv1D models
# train_data = np.reshape(train_data, (train_data.shape[0], train_data.shape[1], 1))
# test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], 1))
train_data = np.reshape(train_data, (train_data.shape[0], train_data.shape[1], train_data.shape[2]))
test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2]))

LSTM with attention

In [ ]:
import tensorflow
from tensorflow import keras
from keras.layers import Input, LSTM, Dense, Dropout, Attention
from keras.models import Model

In [ ]:
import tensorflow as tf

In [ ]:
# Define the input shape based on your EEG data
input_shape = (train_data.shape[1], train_data.shape[2])

# Input layer
input_layer = Input(shape=input_shape)

# LSTM layers
lstm_layer1 = LSTM(64, return_sequences=True)(input_layer)
lstm_dropout1 = Dropout(0.2)(lstm_layer1)

lstm_layer2 = LSTM(64, return_sequences=True)(lstm_dropout1)
lstm_dropout2 = Dropout(0.2)(lstm_layer2)

# Attention mechanism
attention = Attention()([lstm_layer2, lstm_layer2])

# Merge attention and LSTM outputs
merged = tf.keras.layers.Concatenate()([lstm_layer2, attention])

# Fully connected layer
dense_layer = Dense(64, activation='relu')(merged)

# Output layer
output_layer = Dense(1, activation='sigmoid')(dense_layer)


# Create the model
attention_lstm_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
attention_lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_labels = train_labels.reshape((-1, 1))
test_labels = test_labels.reshape((-1, 1))
#to rectify the value error for the label - shape

In [ ]:
# Train the model
attention_history = attention_lstm_model.fit(train_data, train_labels, epochs=10, batch_size=32,
                                             validation_data=(test_data, test_labels), verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model
loss, accuracy = attention_lstm_model.evaluate(test_data, test_labels)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Predict the test labels
test_pred = attention_lstm_model.predict(test_data)
test_pred = (test_pred > 0.5)

# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(test_labels, test_pred))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Get the predicted labels
test_pred = attention_lstm_model.predict(test_data)
test_pred = (test_pred > 0.5)

# Get the confusion matrix
cm = confusion_matrix(test_labels, test_pred)

# Define the class names
class_names = ['Negative', 'Positive']

# Plot the confusion matrix using seaborn
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
# Get training and validation loss curves
train_loss = attention_history.history['loss']
val_loss = attention_history.history['val_loss']

# Plot loss curves
plt.figure(figsize=(5, 4))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Get training and validation accuracy curves
train_acc = attention_history.history['accuracy']
val_acc = attention_history.history['val_accuracy']

# Plot accuracy curves
plt.figure(figsize=(5, 4))
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# Get predicted probabilities for test data
y_pred_proba = attention_lstm_model.predict(test_data)

# Calculate fpr, tpr, and thresholds
fpr, tpr, thresholds = roc_curve(test_labels, y_pred_proba)

# Calculate area under curve (AUC)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(5, 4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

Transformer

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Transformer

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, num_classes):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(input_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

In [ ]:
input_size = 10000
d_model = 128
nhead = 8
num_layers = 4
num_classes = 2

In [ ]:
#Hyerparameters
#d_model
#input size- actual size of vocabulary - not sure
#nhead- number of attention heads
#num_layers- number of stacked transformer layers
#num_classes- number of output classes

In [ ]:
model = TransformerModel(input_size, d_model, nhead, num_layers, num_classes)